In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
dataset.expand(reverse_digit_sequence)

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


----------
## Test different sample rates by cross validation. Then compare the results to determine the optimal sampling rate to use.
#### We try to hold as many parameters constant as possible. We held constan the batch size, num of epochs, n_folds, the random state with which the data is shuffled and the model.
----------

In [7]:
NUM_SAMPLES_TO_TRY = [300, 200, 100, 75, 50, 25, 10]
# model parameters
PARAM_NUM_EPOCHS = 20
PARAM_BATCH_SIZE = 300
# cross validation parameters
N_FOLDS = 3
RANDOM_STATE = 42

In [9]:
import numpy as np
import pandas as pd
from utils.evaluation import cross_validate_model
from models.regularized_deep_gru import NaiveRegularizedDeepGRU

results = {}

for num_samples in NUM_SAMPLES_TO_TRY:
    print("\n\n\n-------------------------------")
    print("Evaluating Spline interpolation using %d samples" % num_samples)
    print("-------------------------------")
    # setup copy of data and evaluate its spline with the currently selected number of samples
    data = dataset.copy()
    data.apply(partial(spline_interpolate_and_resample, num_samples=num_samples))
    x = np.array(data.train_data)
    y = np.array(data.train_labels)
    # setup the model
    mymodel = NaiveRegularizedDeepGRU(x.shape[1:])
    mymodel.batch_size = PARAM_BATCH_SIZE
    mymodel.num_epochs = PARAM_NUM_EPOCHS
    # run cross validation
    scores = cross_validate_model(x, y, mymodel, N_FOLDS, RANDOM_STATE)
    results[num_samples] = scores
    

Using TensorFlow backend.





-------------------------------
Evaluating Spline interpolation using 300 samples
-------------------------------

....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/20
4800/4800 [==============================] - 22s 5ms/step - loss: 2.0968 - categorical_accuracy: 0.2023 - val_loss: 1.9477 - val_categorical_accuracy: 0.2517
Epoch 2/20
4800/4800 [==============================] - 21s 4ms/step - loss: 1.8710 - categorical_accuracy: 0.2777 - val_loss: 1.6881 - val_categorical_accuracy: 0.3308
Epoch 3/20
4800/4800 [==============================] - 21s 4ms/step - loss: 1.6826 - categorical_accuracy: 0.3652 - val_loss: 1.6377 - val_categorical_accuracy: 0.3892
Epoch 4/20
4800/4800 [==============================] - 22s 4ms/step - loss: 1.4365 - categorical_accuracy: 0.4583 - val_loss: 1.2540 - val_categorical_accuracy: 0.5242
Epoch 5/20
4800/4800 [==============================] - 21s 4ms/step - loss: 1.1730 - cat

4800/4800 [==============================] - 21s 4ms/step - loss: 1.3774 - categorical_accuracy: 0.5140 - val_loss: 1.1128 - val_categorical_accuracy: 0.6496
Epoch 6/20
4800/4800 [==============================] - 21s 4ms/step - loss: 1.1698 - categorical_accuracy: 0.5852 - val_loss: 1.0414 - val_categorical_accuracy: 0.6458
Epoch 7/20
4800/4800 [==============================] - 21s 4ms/step - loss: 0.7793 - categorical_accuracy: 0.7400 - val_loss: 0.7138 - val_categorical_accuracy: 0.7767
Epoch 8/20
4800/4800 [==============================] - 21s 4ms/step - loss: 0.5258 - categorical_accuracy: 0.8277 - val_loss: 0.3992 - val_categorical_accuracy: 0.8696
Epoch 9/20
4800/4800 [==============================] - 21s 4ms/step - loss: 0.3195 - categorical_accuracy: 0.9002 - val_loss: 0.3165 - val_categorical_accuracy: 0.9133
Epoch 10/20
4800/4800 [==============================] - 21s 4ms/step - loss: 0.1733 - categorical_accuracy: 0.9458 - val_loss: 0.2581 - val_categorical_accuracy: 0.9

4800/4800 [==============================] - 13s 3ms/step - loss: 0.1441 - categorical_accuracy: 0.9562 - val_loss: 0.2171 - val_categorical_accuracy: 0.9371
Epoch 11/20
4800/4800 [==============================] - 13s 3ms/step - loss: 0.1170 - categorical_accuracy: 0.9658 - val_loss: 0.3302 - val_categorical_accuracy: 0.9075
Epoch 12/20
4800/4800 [==============================] - 13s 3ms/step - loss: 0.1022 - categorical_accuracy: 0.9710 - val_loss: 0.1978 - val_categorical_accuracy: 0.9425
Epoch 13/20
4800/4800 [==============================] - 13s 3ms/step - loss: 0.0762 - categorical_accuracy: 0.9785 - val_loss: 0.1781 - val_categorical_accuracy: 0.9479
Epoch 14/20
4800/4800 [==============================] - 13s 3ms/step - loss: 0.0705 - categorical_accuracy: 0.9779 - val_loss: 0.3422 - val_categorical_accuracy: 0.8946
Epoch 15/20
4800/4800 [==============================] - 13s 3ms/step - loss: 0.0795 - categorical_accuracy: 0.9769 - val_loss: 0.1603 - val_categorical_accuracy:

4800/4800 [==============================] - 6s 1ms/step - loss: 0.0484 - categorical_accuracy: 0.9885 - val_loss: 0.3268 - val_categorical_accuracy: 0.9221
Epoch 16/20
4800/4800 [==============================] - 6s 1ms/step - loss: 0.0301 - categorical_accuracy: 0.9919 - val_loss: 0.2941 - val_categorical_accuracy: 0.9321
Epoch 17/20
4800/4800 [==============================] - 7s 1ms/step - loss: 0.0220 - categorical_accuracy: 0.9946 - val_loss: 0.3105 - val_categorical_accuracy: 0.9279
Epoch 18/20
4800/4800 [==============================] - 7s 1ms/step - loss: 0.0178 - categorical_accuracy: 0.9960 - val_loss: 0.3203 - val_categorical_accuracy: 0.9288
Epoch 19/20
4800/4800 [==============================] - 7s 1ms/step - loss: 0.3037 - categorical_accuracy: 0.9131 - val_loss: 0.2934 - val_categorical_accuracy: 0.9208
Epoch 20/20
2400/2400 [==============================] - 2s 908us/step
categorical_accuracy: 92.58%

....................
Cross validation fold [2]
...................

KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame([[key, np.mean(res), np.std(res)] for key,res in list(results.items())], columns=["Number of Samples", "Categorical Accuracy", "Std Deviation"])
results_df